In [6]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [8]:
subject = 'Acre - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [11]:
data = pd.read_csv('2003_01_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)', 'Acre - Produção de Cimento (t)'], axis=1)
data

,Ano,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - Consumo de Cimento (t)
0,2003,0.691320,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.722906,7.285707e+06,413409.255032,9.318417,6.587220e+06,43.603000
1,2004,0.694888,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.664365,7.395794e+06,415792.317375,9.337292,6.624626e+06,58.142000
2,2005,0.698098,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.632024,7.499439e+06,415180.374027,9.359959,6.658363e+06,80.914000
3,2006,0.700957,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.552716,7.593871e+06,418006.330823,9.385224,6.685538e+06,114.415000
4,2007,0.703473,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.542429,7.683282e+06,420042.950889,9.415550,6.712707e+06,121.872000
5,2008,0.705654,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.514659,7.755114e+06,422290.202070,9.424555,6.724366e+06,119.145000
6,2009,0.707507,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.542386,7.805166e+06,424173.603186,9.409042,6.722105e+06,121.146000
7,2010,0.709040,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.514295,7.991788e+06,416829.675918,9.394384,6.762279e+06,174.705000
8,2011,0.710262,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.500790,8.164220e+06,410997.875281,9.386906,6.795421e+06,178.322000
9,2012,0.711004,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06,173.494000


In [12]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,0.691320,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.722906,7.285707e+06,413409.255032,9.318417,6.587220e+06
1,0.694888,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.664365,7.395794e+06,415792.317375,9.337292,6.624626e+06
2,0.698098,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.632024,7.499439e+06,415180.374027,9.359959,6.658363e+06
3,0.700957,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.552716,7.593871e+06,418006.330823,9.385224,6.685538e+06
4,0.703473,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.542429,7.683282e+06,420042.950889,9.415550,6.712707e+06
5,0.705654,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.514659,7.755114e+06,422290.202070,9.424555,6.724366e+06
6,0.707507,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.542386,7.805166e+06,424173.603186,9.409042,6.722105e+06
7,0.709040,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.514295,7.991788e+06,416829.675918,9.394384,6.762279e+06
8,0.710262,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.500790,8.164220e+06,410997.875281,9.386906,6.795421e+06
9,0.711004,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06


In [13]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      58.142000
1      80.914000
2     114.415000
3     121.872000
4     119.145000
5     121.146000
6     174.705000
7     178.322000
8     173.494000
9     193.672000
10    171.709286
11    151.486629
12    134.223086
13    150.491000
14    148.696000
15    153.806000
16    178.033000
17    189.357000
18    202.321000
19           NaN
Name: Acre - Consumo de Cimento (t), dtype: float64

In [29]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,-2.030084,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-0.817511,-1.693187,0.511437,-0.302580,-2.159170
1,-1.613449,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-0.852793,-1.461874,0.708925,-0.019411,-1.679233
2,-1.238679,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.872285,-1.244099,0.658213,0.320653,-1.246364
3,-0.904868,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.920083,-1.045681,0.892404,0.699688,-0.897688
4,-0.611111,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.926282,-0.857815,1.061182,1.154652,-0.549083
5,-0.356500,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.943019,-0.706882,1.247415,1.289755,-0.399492
6,-0.140130,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.926308,-0.601715,1.403496,1.057021,-0.428509
7,0.038904,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.943239,-0.209591,0.794893,0.837109,0.086954
8,0.181510,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.951378,0.152718,0.311602,0.724917,0.512197
9,0.268191,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.153806,0.491862,0.009045,0.672628,0.926179


In [30]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      58.142000
1      80.914000
2     114.415000
3     121.872000
4     119.145000
5     121.146000
6     174.705000
7     178.322000
8     173.494000
9     193.672000
10    171.709286
11    151.486629
12    134.223086
13    150.491000
14    148.696000
15    153.806000
16    178.033000
Name: Acre - Consumo de Cimento (t), dtype: float64

In [31]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
17,0.972756,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.035224,0.76835,-0.88731,-1.365677,0.306758


In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    189.357
Name: Acre - Consumo de Cimento (t), dtype: float64

In [ ]:
def validation_row_splitter(df): 
    pos = df.shape[0] - 2
    return df.drop(pos), df.iloc[pos:pos + 1]

In [18]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(1, split_factor + 1):
        pos = df.shape[0] - (i*2 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [32]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 5)
    target,target_val = validation_splitter(train_target, 5)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[4051312388, 1782251134, 1628108845, 276095311, 484157264, 377738156, 2473628067, 923432410, 737981757, 1877361596, 3455955187, 4003502984, 1258371583, 2426807839, 1179104927, 290734785, 3693952953, 2502858727, 1695768801, 879889195, 4190023030, 204587325, 1986457217, 2631392089, 1519121871, 2850363278, 2099863429, 2983355011, 1341102195, 4040195525, 739788586, 3064778207, 3307603202, 3035577214, 2357498205, 4146378046, 1596780801, 4107410849, 2539001095, 1398070306, 3231202124, 114029911, 2659757371, 3664410907, 2903217639, 3909605472, 3762342935, 3085007122, 1648708996, 3042673497, 1243504601, 4135758484, 4052952116, 1345052768, 2151665572, 3358394892, 1712689765, 676458020, 1711781014, 3138570086, 942266812, 357240555, 683638831, 1509311884, 3766999450, 271016748, 3493137399, 2104367715, 3120777946, 3179512543, 1801935172, 1173606546, 1096114242, 1296114620, 2712418823, 3749525129, 1745833893, 2112484897, 2987009436, 359812039, 1134529649, 3221714880, 3950921466, 3365750472, 3356168

val_loss: 480.9430236816406


Step: 85 ___________________________________________
val_loss: 27.464500427246094


Step: 86 ___________________________________________
val_loss: 41.75327682495117


Step: 87 ___________________________________________
val_loss: 156.35446166992188


Step: 88 ___________________________________________
val_loss: 104.26934814453125


Step: 89 ___________________________________________
val_loss: 1353.8223876953125


Step: 90 ___________________________________________
val_loss: 83.41121673583984


Step: 91 ___________________________________________
val_loss: 30.55216407775879


Step: 92 ___________________________________________
val_loss: 384.4215087890625


Step: 93 ___________________________________________
val_loss: 189.5633087158203


Step: 94 ___________________________________________
val_loss: 78.61215209960938


Step: 95 ___________________________________________
val_loss: 36.563961029052734


Step: 96 ___________________________________________

In [42]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 477ms/step - loss: 21320.3770 - val_loss: 21921.7793
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 19679.4062 - val_loss: 7805.8247
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 11734.3594 - val_loss: 67536.4688
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 27894.6914 - val_loss: 22253.4062
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 19900.6836 - val_loss: 29825.7090
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 19158.3184 - val_loss: 27036.9922
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 18772.4551 - val_loss: 27282.5801
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 18427.8750 - val_loss: 25351.5723
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 18166.6230 - val_loss: 26194.0312
Epoch 10/10000
1/1 [========

1/1 [==============================] - 0s 13ms/step - loss: 1891.4270 - val_loss: 357.2972
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 2005.2196 - val_loss: 1961.6953
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 1978.7177 - val_loss: 1091.9845
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 1921.1927 - val_loss: 342.1457
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 1756.6803 - val_loss: 276.2675
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 1614.2897 - val_loss: 185.7535
Epoch 158/10000
1/1 [==============================] - 0s 13ms/step - loss: 1407.2491 - val_loss: 213.5713
Epoch 159/10000
1/1 [==============================] - 0s 14ms/step - loss: 773.5607 - val_loss: 31.8061
Epoch 160/10000
1/1 [==============================] - 0s 14ms/step - loss: 393.1656 - val_loss: 278.0446
Epoch 161/10000
1/1 [==============================] -

1/1 [==============================] - 0s 14ms/step - loss: 259.3503 - val_loss: 165.3914
Epoch 308/10000
1/1 [==============================] - 0s 14ms/step - loss: 61.8256 - val_loss: 188.9742
Epoch 309/10000
1/1 [==============================] - 0s 14ms/step - loss: 100.5020 - val_loss: 64.7146
Epoch 310/10000
1/1 [==============================] - 0s 14ms/step - loss: 152.3021 - val_loss: 219.7411
Epoch 311/10000
1/1 [==============================] - 0s 14ms/step - loss: 96.5239 - val_loss: 88.0729
Epoch 312/10000
1/1 [==============================] - 0s 14ms/step - loss: 60.3845 - val_loss: 96.0585
Epoch 313/10000
1/1 [==============================] - 0s 14ms/step - loss: 36.7787 - val_loss: 117.7194
Epoch 314/10000
1/1 [==============================] - 0s 14ms/step - loss: 30.1849 - val_loss: 98.9980
Epoch 315/10000
1/1 [==============================] - 0s 14ms/step - loss: 43.3339 - val_loss: 64.6488
Epoch 316/10000
1/1 [==============================] - 0s 14ms/step - los

1/1 [==============================] - 0s 14ms/step - loss: 54.2822 - val_loss: 179.9379
Epoch 464/10000
1/1 [==============================] - 0s 14ms/step - loss: 39.8301 - val_loss: 205.9037
Epoch 465/10000
1/1 [==============================] - 0s 14ms/step - loss: 65.7197 - val_loss: 180.8612
Epoch 466/10000
1/1 [==============================] - 0s 14ms/step - loss: 54.3677 - val_loss: 166.0830
Epoch 467/10000
1/1 [==============================] - 0s 13ms/step - loss: 47.0004 - val_loss: 169.5196
Epoch 468/10000
1/1 [==============================] - 0s 14ms/step - loss: 39.8155 - val_loss: 107.4249
Epoch 469/10000
1/1 [==============================] - 0s 14ms/step - loss: 36.4354 - val_loss: 199.8743
Epoch 470/10000
1/1 [==============================] - 0s 14ms/step - loss: 48.1762 - val_loss: 26.0989
Epoch 471/10000
1/1 [==============================] - 0s 14ms/step - loss: 50.2097 - val_loss: 273.2236
Epoch 472/10000
1/1 [==============================] - 0s 14ms/step - lo

1/1 [==============================] - 0s 14ms/step - loss: 32.6836 - val_loss: 90.4793
Epoch 542/10000
1/1 [==============================] - 0s 13ms/step - loss: 17.0753 - val_loss: 105.8986
Epoch 543/10000
1/1 [==============================] - 0s 14ms/step - loss: 16.1008 - val_loss: 116.0360
Epoch 544/10000
1/1 [==============================] - 0s 14ms/step - loss: 19.1614 - val_loss: 107.2444
Epoch 545/10000
1/1 [==============================] - 0s 13ms/step - loss: 13.6188 - val_loss: 128.7292
Epoch 546/10000
1/1 [==============================] - 0s 13ms/step - loss: 10.5300 - val_loss: 107.3099
Epoch 547/10000
1/1 [==============================] - 0s 13ms/step - loss: 11.0784 - val_loss: 104.5351
Epoch 548/10000
1/1 [==============================] - 0s 14ms/step - loss: 13.2316 - val_loss: 117.7386
Epoch 549/10000
1/1 [==============================] - 0s 13ms/step - loss: 27.6043 - val_loss: 94.2460
Epoch 550/10000
1/1 [==============================] - 0s 14ms/step - los

1/1 [==============================] - 0s 14ms/step - loss: 19.6701 - val_loss: 150.5347
Epoch 698/10000
1/1 [==============================] - 0s 14ms/step - loss: 32.7925 - val_loss: 136.3633
Epoch 699/10000
1/1 [==============================] - 0s 14ms/step - loss: 53.1008 - val_loss: 123.0323
Epoch 700/10000
1/1 [==============================] - 0s 13ms/step - loss: 16.2937 - val_loss: 135.2646
Epoch 701/10000
1/1 [==============================] - 0s 14ms/step - loss: 32.4132 - val_loss: 134.0045
Epoch 702/10000
1/1 [==============================] - 0s 14ms/step - loss: 17.5189 - val_loss: 141.9285
Epoch 703/10000
1/1 [==============================] - 0s 13ms/step - loss: 32.0748 - val_loss: 123.6052
Epoch 704/10000
1/1 [==============================] - 0s 14ms/step - loss: 30.1290 - val_loss: 124.1416
Epoch 705/10000
1/1 [==============================] - 0s 14ms/step - loss: 30.4781 - val_loss: 123.8124
Epoch 706/10000
1/1 [==============================] - 0s 14ms/step - l

1/1 [==============================] - 0s 14ms/step - loss: 73.2143 - val_loss: 290.5784
Epoch 854/10000
1/1 [==============================] - 0s 14ms/step - loss: 69.2066 - val_loss: 299.8132
Epoch 855/10000
1/1 [==============================] - 0s 14ms/step - loss: 45.2965 - val_loss: 267.1353
Epoch 856/10000
1/1 [==============================] - 0s 14ms/step - loss: 40.2634 - val_loss: 272.5835
Epoch 857/10000
1/1 [==============================] - 0s 14ms/step - loss: 41.9761 - val_loss: 274.5898
Epoch 858/10000
1/1 [==============================] - 0s 14ms/step - loss: 30.0285 - val_loss: 510.3153
Epoch 859/10000
1/1 [==============================] - 0s 14ms/step - loss: 101.3148 - val_loss: 303.0466
Epoch 860/10000
1/1 [==============================] - 0s 14ms/step - loss: 77.4956 - val_loss: 322.3246
Epoch 861/10000
1/1 [==============================] - 0s 14ms/step - loss: 68.0147 - val_loss: 270.5156
Epoch 862/10000
1/1 [==============================] - 0s 14ms/step - 

1/1 [==============================] - 0s 14ms/step - loss: 29.8724 - val_loss: 182.2638
Epoch 932/10000
1/1 [==============================] - 0s 13ms/step - loss: 29.2178 - val_loss: 182.8302
Epoch 933/10000
1/1 [==============================] - 0s 14ms/step - loss: 28.6537 - val_loss: 183.6615
Epoch 934/10000
1/1 [==============================] - 0s 13ms/step - loss: 28.1097 - val_loss: 184.2159
Epoch 935/10000
1/1 [==============================] - 0s 14ms/step - loss: 22.6622 - val_loss: 163.3073
Epoch 936/10000
1/1 [==============================] - 0s 14ms/step - loss: 28.7404 - val_loss: 184.8535
Epoch 937/10000
1/1 [==============================] - 0s 14ms/step - loss: 26.9654 - val_loss: 183.9264
Epoch 938/10000
1/1 [==============================] - 0s 14ms/step - loss: 26.4458 - val_loss: 184.3945
Epoch 939/10000
1/1 [==============================] - 0s 14ms/step - loss: 25.6909 - val_loss: 193.5958
Epoch 940/10000
1/1 [==============================] - 0s 14ms/step - l

Epoch 1086/10000
1/1 [==============================] - 0s 14ms/step - loss: 102.1109 - val_loss: 274.2409
Epoch 1087/10000
1/1 [==============================] - 0s 14ms/step - loss: 73.4229 - val_loss: 261.8237
Epoch 1088/10000
1/1 [==============================] - 0s 13ms/step - loss: 83.8286 - val_loss: 277.1201
Epoch 1089/10000
1/1 [==============================] - 0s 14ms/step - loss: 69.0088 - val_loss: 267.7267
Epoch 1090/10000
1/1 [==============================] - 0s 14ms/step - loss: 94.1925 - val_loss: 254.1801
Epoch 1091/10000
1/1 [==============================] - 0s 13ms/step - loss: 282.6812 - val_loss: 409.0792
Epoch 1092/10000
1/1 [==============================] - 0s 13ms/step - loss: 317.6976 - val_loss: 238.5594
Epoch 1093/10000
1/1 [==============================] - 0s 14ms/step - loss: 248.8496 - val_loss: 314.1864
Epoch 1094/10000
1/1 [==============================] - 0s 13ms/step - loss: 231.3937 - val_loss: 303.6410
Epoch 1095/10000
1/1 [===================

In [43]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 36ms/step
[189.357](test_target) - [[174.21902]](prediction) = 15.137975341796874


In [44]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [45]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,-1.719426,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,1.890129,-1.579970,-1.390443,-1.728144,-1.794558
1,-1.108757,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,1.083550,-1.069243,-0.701037,-1.203190,-1.105592
2,-0.559449,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,0.637946,-0.588405,-0.878069,-0.572763,-0.484192
3,-0.070177,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.454770,-0.150307,-0.060537,0.129913,0.016343
4,0.360389,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.596500,0.264492,0.528645,0.973348,0.516778
5,0.733577,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.979118,0.597744,1.178761,1.223809,0.731521
6,1.050714,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,-0.597097,0.829948,1.723617,0.792356,0.689866
7,1.313129,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,-0.984140,1.695740,-0.400936,0.384671,1.429832


train_target:


,Acre - Consumo de Cimento (t)
0,58.142
1,80.914
2,114.415
3,121.872
4,119.145
5,121.146
6,174.705
7,178.322


test_input:


...

,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
17,0.972756,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.035224,0.76835,-0.88731,-1.365677,0.306758


test_target:


,Acre - Consumo de Cimento (t)
17,189.357


1/1 [==============================] - 0s 34ms/step
Error: 15.137975341796874




[147.79373168945312,
 165.78704833984375,
 164.40052795410156,
 181.2442626953125,
 162.22825622558594,
 141.52113342285156,
 172.578369140625,
 156.54327392578125,
 150.16265869140625,
 174.21902465820312]

In [46]:
display(targets)
display(predictions)

[173.494,
 193.672,
 171.7092857142857,
 151.48662857142855,
 134.2230857142857,
 150.491,
 148.696,
 153.80599999999998,
 178.033,
 189.357]

[147.79373168945312,
 165.78704833984375,
 164.40052795410156,
 181.2442626953125,
 162.22825622558594,
 141.52113342285156,
 172.578369140625,
 156.54327392578125,
 150.16265869140625,
 174.21902465820312]

In [47]:
mae = mean_absolute_error(predictions, targets)
mae

19.725460866001676

In [48]:
porcentage = mae/np.mean(targets)
porcentage

0.11991394887925892